In [1]:
# Se importan todas las librerias que usaremos
import requests
import pandas as pd
import numpy as np
import sqlalchemy
import pymysql

# Se importa la clave de la API
from keys import *

In [2]:
# Se declara la ciudad y el país a investigar y se hace la petición a la API
city = "Barcelona"
country = "ES"

# Se llama a la API de OpenWeatherMap
response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')

In [3]:
# Recuperar datos en formato JSON
data = response.json()

# Extraer la lista de pronósticos
forecast_list = data.get('list', [])

# Preparar listas para cada columna
times = []
temperatures = []
humidities = []
weather_statuses = []
wind_speeds = []
rain_volumes = []
snow_volumes = []

# Rellenar las listas con los datos del pronóstico
for entry in forecast_list:
    times.append(entry.get('dt_txt', np.nan))
    temperatures.append(entry.get('main', {}).get('temp', np.nan))
    humidities.append(entry.get('main', {}).get('humidity', np.nan))
    weather_statuses.append(entry.get('weather', [{}])[0].get('main', np.nan))
    wind_speeds.append(entry.get('wind', {}).get('speed', np.nan))
    rain_volumes.append(entry.get('rain', {}).get('3h', 0))  # 0 en lugar de NaN
    snow_volumes.append(entry.get('snow', {}).get('3h', 0))  # 0 en lugar de NaN

# Crear el DataFrame
df = pd.DataFrame({
    'weather_datetime': times,
    'temperature': temperatures,
    'humidity': humidities,
    'weather_status': weather_statuses,
    'wind': wind_speeds,
    'rain_qty': rain_volumes,
    'snow': snow_volumes,
    'municipality_iso_country': 'Barcelona,ES'  # Formato corregido
})

In [4]:
# Imprime los valores que se encuentran en el DataFrame
print(df.head())

      weather_datetime  temperature  humidity weather_status  wind  rain_qty  \
0  2025-11-07 00:00:00        14.73        68         Clouds  3.20       0.0   
1  2025-11-07 03:00:00        14.67        69         Clouds  2.70       0.0   
2  2025-11-07 06:00:00        15.11        71         Clouds  2.83       0.0   
3  2025-11-07 09:00:00        15.22        63          Clear  2.81       0.0   
4  2025-11-07 12:00:00        19.18        54         Clouds  0.81       0.0   

   snow municipality_iso_country  
0     0             Barcelona,ES  
1     0             Barcelona,ES  
2     0             Barcelona,ES  
3     0             Barcelona,ES  
4     0             Barcelona,ES  


In [ ]:
# Conexión a MySQL
schema = "gans"
host = "127.0.0.1"
user = "root"
password = "EvanManx8"  # Asegúrate de que esta contraseña es correcta
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
# Busca insertar los datos en la tabla weather_data
df.to_sql('weather_data', if_exists='append', con=con, index=False)

# Imprime un mensaje de confirmación
print("Datos insertados correctamente en la base de datos")

Datos insertados correctamente en la base de datos
